In [5]:
import pandas as pd
############################################
# ~ LABELED DATA FOR ROUND 1 and ROUND 2 ~ #
############################################
labeled_datafile= '/home/ec2-user/SageMaker/serperi/system/sessions/scal/One_second_round/data/labeled_data2022-11-23_00-18.csv'
df = pd.read_csv(labeled_datafile, sep=';',header=None)
df.head()
df = df.rename(columns={0:'id',1:'label'} )

first_round=df.iloc[:315,:]
second_round=df.iloc[315:,:]
relevant_in_first_round = len([label for label in first_round['label'] if label=='R'])
relevant_in_second_round = len([label for label in second_round['label'] if label=='R'])

print(f'First round:       {len(first_round):10} (relevant={relevant_in_first_round:3})')
print(f'Second round:      {len(second_round):10} (relevant={relevant_in_second_round:3})')
print('--')

assert first_round.shape[0]==len(set(first_round['id']))
assert second_round.shape[0]==len(set(second_round['id']))

########################################################
# ~ LABELED DATA FROM EVALUATION ROUND 1 and ROUND 2 ~ #
########################################################
evaluation50_file = 'evaluation50.csv'
evaluation100_file = 'evaluation100.csv'

first_evaluation = pd.read_csv(evaluation100_file, sep=';')
second_evaluation = pd.read_csv(evaluation50_file, sep=';')

relevant_in_first_evaluation = len([label for label in first_evaluation['label'] if label=='R'])
relevant_in_second_evaluation = len([label for label in second_evaluation['label'] if label=='R'])
                                
print(f'First evaluation:  {len(first_evaluation):10} (relevant={relevant_in_first_evaluation:3})')
print(f'Second evaluation: {len(second_evaluation):10} (relevant={relevant_in_second_evaluation:3})')
print('--')


assert first_evaluation.shape[0]==len(set(first_evaluation['id']))
assert second_evaluation.shape[0]==len(set(second_evaluation['id']))

##############
# ~ SHARED ~ #
##############
R1 = set(first_round['id'])
R2 = set(second_round['id'])

E1 = set(first_evaluation['id'])
E2 = set(second_evaluation['id'])
print(f'Shared between first evaluation and second round:      {len(E1.intersection(R2))}')
print(f'Shared between first evaluation and second evaluation: {len(E1.intersection(E2))}')
print('--')
###########
# ~ ALL ~ #
###########
all_data_with_dups = first_round.append(first_evaluation).append(second_round).append(second_evaluation)
# all_data.shape

d = {}
conflicts=0
for i in range(all_data_with_dups.shape[0]):
    id_, label = all_data_with_dups.iloc[i,]
    if id_ in d and d[id_]!=label: # We already saw this item with a different label.
        d[id_]='U'
        conflicts+=1
    else:
        d[id_]=label

print(f'All data with no dups {len(d)} - conflicts={conflicts}')

items = d.items()
all_data = pd.DataFrame({'id': [key  for key,value in items ],
                         'label':[value for key,value in items]
                        })

conflicting_data = {1237561744:'R',
                    1237268511:'R',
                    1444938912:'R',
                    1444939955:'I',
                    1143705076:'R',
                    1237792063:'I',
                   }
for id_ in conflicting_data:
    all_data.loc[all_data['id']==id_,'label']=conflicting_data[id_]

all_data.head()

all_data.to_csv('/home/ec2-user/SageMaker/mariano/datasets/multiculturalism/files/labeled_data.csv', index=False)

First round:              315 (relevant= 12)
Second round:             290 (relevant= 49)
--
First evaluation:         100 (relevant= 29)
Second evaluation:         50 (relevant= 14)
--
Shared between first evaluation and second round:      16
Shared between first evaluation and second evaluation: 2
--
All data with no dups 737 - conflicts=6
